<a href="https://colab.research.google.com/github/DesmondChoy/create_adventure/blob/main/prompt_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install claudette -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.5/863.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00


In [2]:
import os
import random
import json
from claudette.core import Chat, Client, models
from google.colab import userdata

os.environ['ANTHROPIC_LOG'] = 'debug'
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')


# Purpose: Build a MVP

This is a web-based "Choose your own adventure" RPG, but using LLMs and with an educational twist. Through the adventure, the reader is offered closed-ended player agency choices (strategic decisions, unique equipment, companion, etc).


MVP: 5 pages.

How the workflow looks like in my mind:

1. Intialization
- Pre-define data: educational trivia category, q&a pairs
- Begin by asking what type of adventure to embark on: choose_topic
- choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
- educational_trivia = random.choice(["Singapore history", "Farm animals", "Human body"])

2. Page I & II
- A function start_adventure()  takes {choose_topic}  and {educational_trivia} as arguments
- start_adventure() produces Page I and II
- Page I sets the context and character development, with no choices. The plot/narrative must be related to {educational_trivia}
- Page II concludes with a Player agency choice. Answer is saved as user_choice

3. Page III
- A function continue_adventure() takes two arguments, {story_so_far} and {user_choice}
- continue_adventure() continues to build the plot and is shaped by {user_choice}
- Page III concludes with a trivia question related to {educational_trivia}. Answer is saved as user_answer

4. Page IV
- continue_adventure() is used again but it will need to take an additional argument {user_answer} together with {story_so_far}
- continue_adventure() continues to build the plot and is shaped by whether {user_answer} was correct or incorrect. If incorrect, there should be a lesson educating the reader on the correct answer.
- Page IV concludes with a Player agency choice. Answer is saved as user_choice

5. Page V
- A function concludes adventure() takes two arguments, {story_so_far} and {user_choice}
- concludes adventure() reveals the plot and is shaped by {user_choice}. The plot is also related to {educational_trivia}.
- There will be a happy ending but there is also a hint of more adventures to come.

Some comments
- Persistence: The story state between pages is saved as string variables for now. I'll think of a better solution later down the road.

- Modularity: The focus is on shipping a MVP and refactoring will be done later. I'm open to simple, intuitive suggestions on how the functions should like like e.g. class structures

- Story generation: Claude Sonnet 3.5 will be used to generate the narrative dynamically.



To be included later:
- Difficulty level (easy/medium/hard) - this will define maths qns.
- Rendered images



# Intialization

In [3]:
import random
from claudette.core import Chat, Client, models

# Initialize the Claude client
client = Client(models[-1])
chat = Chat(cli=client)

choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
trivia_topic = ["Singapore history", "Farm animals", "Human body"]

trivia_questions = {
    "Singapore history": [
        {
            "question": "What animal is the symbol of Singapore?",
            "choices": [
                "Lion",
                "Tiger",
                "Elephant"
            ],
            "correct_answer": "Lion"
        },
        {
            "question": "What color is the top half of Singapore's flag?",
            "choices": [
                "Blue",
                "Red",
                "White"
            ],
            "correct_answer": "Red"
        }
    ],
    "Farm animals": [
        {
            "question": "Which farm animal gives us milk?",
            "choices": [
                "Chicken",
                "Cow",
                "Pig"
            ],
            "correct_answer": "Cow"
        },
        {
            "question": "What do we call a baby sheep?",
            "choices": [
                "Calf",
                "Kitten",
                "Lamb"
            ],
            "correct_answer": "Lamb"
        }
    ],
    "Human body": [
        {
            "question": "How many fingers do you have on one hand?",
            "choices": [
                "Four",
                "Five",
                "Six"
            ],
            "correct_answer": "Five"
        },
        {
            "question": "Which organ helps you breathe?",
            "choices": [
                "Heart",
                "Brain",
                "Lungs"
            ],
            "correct_answer": "Lungs"
        }
    ]
}

In [4]:
# Import necessary libraries
import random
from claudette.core import Chat, Client, models

# Initialize the Claudette client and chat
client = Client(models[-1])  # Using the latest Claude model
chat = Chat(cli=client)

# Define the data
choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
trivia_topic = ["Singapore history", "Farm animals", "Human body"]
trivia_questions = {
    "Singapore history": [
        {
            "question": "What animal is the symbol of Singapore?",
            "choices": [
                "Lion",
                "Tiger",
                "Elephant"
            ],
            "correct_answer": "Lion"
        },
        {
            "question": "What color is the top half of Singapore's flag?",
            "choices": [
                "Blue",
                "Red",
                "White"
            ],
            "correct_answer": "Red"
        }
    ],
    "Farm animals": [
        {
            "question": "Which farm animal gives us milk?",
            "choices": [
                "Chicken",
                "Cow",
                "Pig"
            ],
            "correct_answer": "Cow"
        },
        {
            "question": "What do we call a baby sheep?",
            "choices": [
                "Calf",
                "Kitten",
                "Lamb"
            ],
            "correct_answer": "Lamb"
        }
    ],
    "Human body": [
        {
            "question": "How many fingers do you have on one hand?",
            "choices": [
                "Four",
                "Five",
                "Six"
            ],
            "correct_answer": "Five"
        },
        {
            "question": "Which organ helps you breathe?",
            "choices": [
                "Heart",
                "Brain",
                "Lungs"
            ],
            "correct_answer": "Lungs"
        }
    ]
}

# Function to get user input
def get_user_input(prompt, options):
    while True:
        print(prompt)
        for i, option in enumerate(options, 1):
            print(f"{i}. {option}")
        try:
            choice = int(input("Enter the number of your choice: "))
            if 1 <= choice <= len(options):
                return options[choice - 1]
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# Function to generate story content
def generate_story(prompt):
    response = chat(prompt)
    return response.content[0].text

# Main game flow
def run_game():
    # Page I & II
    chosen_topic = get_user_input("Choose a topic for your adventure:", choose_topic)
    trivia_topic_chosen = random.choice(trivia_topic)

    prompt_page1 = f"""Create the first page of a children's story that sets the context and introduces the main character.
    The story should be related to {chosen_topic} and subtly incorporate elements of {trivia_topic_chosen}.
    End the page with a cliffhanger that leads to the next part of the story."""

    page1 = generate_story(prompt_page1)
    print("\nPage I:\n", page1)

    prompt_page2 = f"""Continue the story from the previous page. Develop the plot and character further,
    keeping the themes of {chosen_topic} and {trivia_topic_chosen} in mind.
    End this page with a choice for the reader that will affect the story.
    Present two clear options for the reader to choose from."""

    page2 = generate_story(prompt_page2)
    print("\nPage II:\n", page2)

    # Extract choices from page2
    choice1_options = page2.split("Option 1:")[-1].split("Option 2:")
    choice1_option1 = choice1_options[0].strip()
    choice1_option2 = choice1_options[1].strip()

    choice1 = get_user_input("Make your choice:", [choice1_option1, choice1_option2])

    # Page III
    prompt_page3 = f"""Continue the story based on the reader's choice: {choice1}.
    Develop the plot further, incorporating elements of {chosen_topic} and {trivia_topic_chosen}.
    End this page by presenting a situation where the main character needs to answer a question related to {trivia_topic_chosen}."""

    page3 = generate_story(prompt_page3)
    print("\nPage III:\n", page3)

    # Select a trivia question
    trivia_question = random.choice(trivia_questions[trivia_topic_chosen])
    user_answer = get_user_input(trivia_question["question"], trivia_question["choices"])
    is_correct = user_answer == trivia_question["correct_answer"]

    # Page IV
    prompt_page4 = f"""Continue the story, taking into account that the character {'correctly' if is_correct else 'incorrectly'}
    answered a question about {trivia_topic_chosen}. If the answer was incorrect, include a subtle lesson about the correct information.
    Develop the plot further and end this page with another choice for the reader that will affect the story's conclusion.
    Present two clear options for the reader to choose from."""

    page4 = generate_story(prompt_page4)
    print("\nPage IV:\n", page4)

    # Extract choices from page4
    choice2_options = page4.split("Option 1:")[-1].split("Option 2:")
    choice2_option1 = choice2_options[0].strip()
    choice2_option2 = choice2_options[1].strip()

    choice2 = get_user_input("Make your choice:", [choice2_option1, choice2_option2])

    # Page V
    prompt_page5 = f"""Conclude the story, taking into account the reader's choices: {choice1} and {choice2}.
    Bring the adventure to a satisfying and happy ending, but also hint at the possibility of future adventures.
    Make sure the conclusion relates to the original themes of {chosen_topic} and {trivia_topic_chosen}."""

    page5 = generate_story(prompt_page5)
    print("\nPage V:\n", page5)

# Run the game
run_game()

Choose a topic for your adventure:
1. Magical Realms
2. Animal Adventures
3. Mystery and Detectives
4. Inventions and Gadgets
Enter the number of your choice: 3

Page I:
 Here is the first page of a children's story that sets the context, introduces the main character, and incorporates elements of mystery, detectives, and farm animals, ending with a cliffhanger:

The Case of the Missing Eggs

It was a bright and sunny morning on Farmer Brown's farm. Daisy the cow was chewing her cud, the chickens were clucking happily as they pecked at the ground, and Farmer Brown was tending to his vegetable garden.

But something was not quite right. As Farmer Brown made his rounds, he noticed that the chicken coop was strangely quiet. "That's odd," he thought to himself. "The hens are usually making a ruckus by now."

Farmer Brown approached the coop and peeked inside. To his dismay, the nests were empty! "My goodness, where are all the eggs?" he exclaimed.

Just then, a small voice piped up from be